In [1]:
#install gensim library for topic modeling algorithm 
!python -m pip install -U gensim 

Requirement already up-to-date: gensim in /opt/anaconda3/lib/python3.8/site-packages (3.8.3)


In [4]:
import os.path #to work in iOS
from gensim import corpora # gensim model to create a corpus dict
from gensim.models import LsiModel # gensim Latent Sentiment Indexing model - generates topics
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords # remove stopwords to process data into corpus
from nltk.stem.porter import PorterStemmer #for stemming and lemmatization 
from gensim.models.coherencemodel import CoherenceModel# to calculate coherence values to determine optimal topic numbers
import matplotlib.pyplot as plt 

In [5]:
# Create data load function for loading covid_cleandf2.csv. 
# """
#     Input  : path and file_name
#     Purpose: loading text file
#     Output : list of paragraphs/documents and
#              title(initial 100 words considred as title of document)
# with open( os.path.join(path, file_name) ,"r", encoding='utf_8') as fin:


#     """

def load_data(path, file_name):
    documents_list = []
    titles=[]
    with open( os.path.join(path, file_name) ,"r") as fin:
        for line in fin.readlines():
            text = line.strip()
            documents_list.append(text)
    print("Total Number of Documents:",len(documents_list))
    titles.append( text[0:min(len(text),100)] )
    return documents_list,titles

In [6]:
documents_list,titles = load_data('/Users/tamaralottering/Downloads/GitHub/ds4d-project/Clean Data','UK_cleandf.txt')

Total Number of Documents: 2683


In [18]:
# After data loading function, you need to preprocess the text. 
# Following steps are taken to preprocess the text:

# Tokenize the text articles
# Remove stop words
# Perform stemming on text artcle

def preprocess_data(documents_list):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in documents_list:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

In [19]:
doc_clean = preprocess_data(documents_list)

In [20]:
# Prepare Corpus
# Next step is to prepare corpus. Here, you need to create a document-term matrix and dictionary of terms.

def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

In [21]:
 dictionary,doc_term_matrix = prepare_corpus(doc_clean)

In [22]:
# Create an LSA model using Gensim
# After corpus creation, you can generate a model using LSA.

def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix = prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

In [23]:
# Determine the number of topics
# Another extra step needs to be taken to optimize results by identifying an optimum amount of topics. Here, you will generate coherence scores to determine an optimum number of topics.

def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop = number_of_topics, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for number_of_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [24]:
#Let's plot coherence score values to have a sense of optimal no. of topics
def plot_graph(doc_clean,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix,doc_clean,
                                                            stop, start, step)
    # Show graph
    x = range(start, stop, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [ ]:
start,stop,step=2,12,1
plot_graph(doc_clean,start,stop,step)

In [ ]:
# LSA Model
number_of_topics=4
words=10
document_list,titles=load_data('/Users/tamaralottering/Downloads/GitHub/ds4d-project/Clean Data','UK_cleandf.txt')
text=preprocess_data(documents_list)
model=create_gensim_lsa_model(doc_clean,number_of_topics,words)

#### Topics: 
- Topic 1:"said", "coronaviru", "case" + 0.201*"peopl", "countri", "new","health", "report", "death", "covid19"( Number of deaths due to and new cases of Covid-19)
- Topic 2: gmt","bst", "death", "close","covid19","lockdown","pandem","march","travel","announc" (Lockdown and travel restrictions due to Covid-19 pandemic)
- Topic 3: "bst", "blocktim", coronaviru", "say","countri", "publishedtim", "work", "care","updatedtimeupd, need" (unclear)
- Topic 4:"case", trump", new", "juli", "us", "victoria","infect","test", presid", covid19" (President Trump, testing and new cases)